Importing important libraries

In [2]:
import pandas as pd
import numpy as np
!conda install -c conda-forge geopy --yes
!conda install -c conda-forge folium=0.5.0 --yes 
from bs4 import BeautifulSoup
import folium
import xml
import requests
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print("Libraries imported")

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

Extracting the content of the Wikipedia page "List of all US cities by population"- 

In [4]:
link='https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page=requests.get(link)
soup=BeautifulSoup(page.text)

Finding the data in the webpage that contains required data and columns like Rank, Cities, Population, Location.

In [5]:
table=soup.find_all('table')[4]

Extracting the table from the wikipedia page and saving it into the newly created Dataframe with specified column names.

In [6]:
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Rank","City", "State", "w1", "w2", "w3", "Pop_sq.area", "w4", "Population density in Sq-mi", "Population density in Sq-Km2", "Location"])
#df=pd.DataFrame(res, columns=["place", "Location", "Visitors"])
df.head()

,Rank,City,State,w1,w2,w3,Pop_sq.area,w4,Population density in Sq-mi,Population density in Sq-Km2,Location
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


Dropping those columns which are not required-

In [8]:
df.drop(columns=["w1", "w2", "w3","Pop_sq.area", "w4","Population density in Sq-mi" ], inplace=True)

Listing the Dataframe after deletion of columns

In [9]:
df

,Rank,City,State,Population density in Sq-Km2,Location,Radius
0,1,New York[d],New York,"10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,17.363755
1,2,Los Angeles,California,"3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,21.649480
2,3,Chicago,Illinois,"4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,15.076472
3,4,Houston[3],Texas,"1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,25.248762
4,5,Phoenix,Arizona,"1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,22.750824
5,6,Philadelphia[e],Pennsylvania,"4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W﻿...,11.584472
6,7,San Antonio,Texas,"1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W﻿...,21.470911
7,8,San Diego,California,"1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°...,18.033303
8,9,Dallas,Texas,"1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W﻿...,18.463477
9,10,San Jose,California,"2,231/km2",37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°...,13.322913


Finding the "Radius" of each city using Pop_sq.area column

In [7]:
new=df["Pop_sq.area"].str.split("s", n=1, expand=True)
new=new[0].str.replace(u'\xa0',u'')
df["Pop_sq.area"]=new.str.replace(',', '')
df["Pop_sq.area"]=df["Pop_sq.area"].astype(float)
df["Radius"]=np.sqrt(df["Pop_sq.area"])

Splitting the Location in coordinates [that is into Longitude and Latitude]

In [10]:
df["Location"]=df["Location"].str.split('/', n=2, expand=True)[1]

In [11]:
df.head()

,Rank,City,State,Population density in Sq-Km2,Location,Radius
0,1,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿,17.363755
1,2,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿,21.649480
2,3,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿,15.076472
3,4,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿,25.248762
4,5,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿,22.750824


In [12]:
new=df["Location"].str.split(" ", n=0, expand=False)
k=df.copy(deep=True)
df.head()

,Rank,City,State,Population density in Sq-Km2,Location,Radius
0,1,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿,17.363755
1,2,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿,21.649480
2,3,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿,15.076472
3,4,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿,25.248762
4,5,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿,22.750824


In [13]:
Latitude=[]
Longitude=[]
for i in range(len(new)):
    Latitude.append(new[i][1][:-2])
    Longitude.append(new[i][2][:-3])
k["Latitude"]=Latitude
k["Longitude"]=Longitude
k["Latitude"] = k["Latitude"].str.replace(u'\ufeff',u'')
k.drop(columns=["Location"], inplace=True)
k.head()
df=k.copy(deep=True)

In [14]:
df

,Rank,City,State,Population density in Sq-Km2,Radius,Latitude,Longitude
0,1,New York[d],New York,"10,933/km2",17.363755,40.6635,73.9387
1,2,Los Angeles,California,"3,276/km2",21.649480,34.0194,118.4108
2,3,Chicago,Illinois,"4,600/km2",15.076472,41.8376,87.6818
3,4,Houston[3],Texas,"1,395/km2",25.248762,29.7866,95.3909
4,5,Phoenix,Arizona,"1,200/km2",22.750824,33.5722,112.0901
5,6,Philadelphia[e],Pennsylvania,"4,511/km2",11.584472,40.0094,75.1333
6,7,San Antonio,Texas,"1,250/km2",21.470911,29.4724,98.5251
7,8,San Diego,California,"1,670/km2",18.033303,32.8153,117.1350
8,9,Dallas,Texas,"1,493/km2",18.463477,32.7933,96.7665
9,10,San Jose,California,"2,231/km2",13.322913,37.2967,121.8189


Taking Longitude as negative and multipying radius with 1000

In [15]:
df['Longitude'] = -df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)
df['Radius'] = df['Radius']* 1000
df.head()

,Rank,City,State,Population density in Sq-Km2,Radius,Latitude,Longitude
0,1,New York[d],New York,"10,933/km2",17363.755354,40.6635,-73.9387
1,2,Los Angeles,California,"3,276/km2",21649.480363,34.0194,-118.4108
2,3,Chicago,Illinois,"4,600/km2",15076.471736,41.8376,-87.6818
3,4,Houston[3],Texas,"1,395/km2",25248.762346,29.7866,-95.3909
4,5,Phoenix,Arizona,"1,200/km2",22750.824161,33.5722,-112.0901


Getting Data of Per Captia Income from the wikipedia page

In [16]:
link1='https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income'
page1=requests.get(link1)
soup1=BeautifulSoup(page1.text)

Extracting Data from the webpage

In [17]:
table=soup1.find_all('table')[1]
table_rows=table.find_all('tr')

Saving the Data into newly created Dataframe named as df_state

In [18]:
result=[]
for tr in table_rows:
    td=tr.find_all('td')
    row=[tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        result.append(row)
df_state=pd.DataFrame(result, columns=["Rank", "Country-equivalent", "State","Per Capita Income","w1","w2","Population","w3" ])
df_state.head()

,Rank,Country-equivalent,State,Per Capita Income,w1,w2,Population,w3
0,1,New York County,New York,"$62,498","$69,659","$84,627","1,605,272","736,192"
1,2,Arlington,Virginia,"$62,018","$103,208","$139,244","214,861","94,454"
2,3,Falls Church City,Virginia,"$59,088","$120,000","$152,857","12,731","5,020"
3,4,Marin,California,"$56,791","$90,839","$117,357","254,643","102,912"
4,5,Alexandria City,Virginia,"$54,608","$85,706","$107,511","143,684","65,369"


In [19]:
df_state.drop(columns=['Rank', "w1", "w2","w3"], inplace=True, axis=1)
df_state.head()

,Country-equivalent,State,Per Capita Income,Population
0,New York County,New York,"$62,498","1,605,272"
1,Arlington,Virginia,"$62,018","214,861"
2,Falls Church City,Virginia,"$59,088","12,731"
3,Marin,California,"$56,791","254,643"
4,Alexandria City,Virginia,"$54,608","143,684"


Creating Map of USA Cities

In [20]:
map_usa=folium.Map(location=[37.0902, -95.7129], zoom_start=5)

for lat, lng, state, city in zip(df['Latitude'], df['Longitude'], df['State'], df['City']):
    folium.CircleMarker(
    [lat, lng],
    radius=4,
    color='red',
    fill=True,
    fill_color='#cc3139',
    fill_opacity=0.3
    ).add_to(map_usa)
map_usa

Defining Foursquare credentials and Version

In [21]:
CLIENT_ID='AHNKJN3DPM53FI2MGY00U54G3FDUHNVIXH1NBIHCBFIRD2FY'
CLIENT_SECRET='BZNDA4BHNDSWKIQW3MOF3JZPW2OWIMRVSLUQCOE3HAODDULY'
VERSION='20190626'
LIMIT=20
print('Your Credentials')
print('Your Client Id is'+CLIENT_ID)
print('your client secret is' +CLIENT_SECRET)

Your Credentials
Your Client Id isAHNKJN3DPM53FI2MGY00U54G3FDUHNVIXH1NBIHCBFIRD2FY
your client secret isBZNDA4BHNDSWKIQW3MOF3JZPW2OWIMRVSLUQCOE3HAODDULY


Creating a Function that extracts necessary columns into a dataframe from the json files that we get when we search using Foursquare API out of the values that we have in the dataframe that we got from the webpage

In [22]:

def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng,radius in zip(names, latitudes, longitudes,radius):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       # print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:

df_venues = getNearbyVenues(names = df['City'], latitudes = df['Latitude'],longitudes = df['Longitude'], radius = df['Radius'])
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum
3,New York[d],40.6635,-73.9387,Prospect Park Boathouse & Audubon Center,40.660884,-73.964949,Building
4,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater


Assigning weights to some of the categories that Ramesh wants to consider(as there are subsets of what he initially thought of)

In [25]:
k = df_venues.copy(deep = True)
weights_dict={'Movie Theater':3,'Beach':3,'Concert Hall':2.5,'Playground':3,'Coffee Shop':3.5,'Food Court':4,'Nightclub':4,'Toy / Game Store':4.5,'Theme Park Ride / Attraction':4,'Pub':4}
data = df_venues['Venue Category']
#allVenues = data.astype(list)

In [26]:
weights = []
for i in data:
    if i in weights_dict.keys():
        weights.append(weights_dict[i])
    else :
        weights.append(0)
df_venues['weights'] = weights;
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden,0.0
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar,0.0
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum,0.0
3,New York[d],40.6635,-73.9387,Prospect Park Boathouse & Audubon Center,40.660884,-73.964949,Building,0.0
4,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater,0.0


Dropping those rows which are not assigned any weight

In [27]:
df_venues.drop(df_venues[df_venues.weights<1.0].index, inplace=True)
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
22,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,34.027115,-118.387637,Coffee Shop,3.5
30,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,33.980027,-118.408020,Coffee Shop,3.5
59,Chicago,41.8376,-87.6818,Sawada Coffee,41.883730,-87.648726,Coffee Shop,3.5
77,Houston[3],29.7866,-95.3909,White Oak Music Hall,29.785994,-95.367173,Concert Hall,2.5
99,Phoenix,33.5722,-112.0901,Dutch Bros. Coffee,33.509570,-112.074119,Coffee Shop,3.5


In [62]:
df_venues.reset_index

<bound method DataFrame.reset_index of                  City  Latitude  Longitude  \
22        Los Angeles   34.0194  -118.4108   
30        Los Angeles   34.0194  -118.4108   
59            Chicago   41.8376   -87.6818   
77         Houston[3]   29.7866   -95.3909   
99            Phoenix   33.5722  -112.0901   
100   Philadelphia[e]   40.0094   -75.1333   
138       San Antonio   29.4724   -98.5251   
157         San Diego   32.8153  -117.1350   
222   Jacksonville[f]   30.3369   -81.6616   
224   Jacksonville[f]   30.3369   -81.6616   
256        Fort Worth   32.7815   -97.3467   
264          Columbus   39.9852   -82.9848   
272          Columbus   39.9852   -82.9848   
278          Columbus   39.9852   -82.9848   
316   Indianapolis[h]   39.7767   -86.1459   
322           Seattle   47.6205  -122.3509   
328           Seattle   47.6205  -122.3509   
335           Seattle   47.6205  -122.3509   
338           Seattle   47.6205  -122.3509   
341           Seattle   47.6205  -122.350

In [28]:
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
22,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,34.027115,-118.387637,Coffee Shop,3.5
30,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,33.980027,-118.408020,Coffee Shop,3.5
59,Chicago,41.8376,-87.6818,Sawada Coffee,41.883730,-87.648726,Coffee Shop,3.5
77,Houston[3],29.7866,-95.3909,White Oak Music Hall,29.785994,-95.367173,Concert Hall,2.5
99,Phoenix,33.5722,-112.0901,Dutch Bros. Coffee,33.509570,-112.074119,Coffee Shop,3.5


Copying only those columns which are necessary example City, weights to group all the venues by city and calculating the means for each city

In [29]:
citywise_venues_weights=df_venues[['City', 'weights']].copy()
citywise_venues_weights_means=citywise_venues_weights.groupby(['City']).mean()
citywise_venues_weights_means=citywise_venues_weights_means.reset_index(drop=False)
citywise_venues_weights_means

,City,weights
0,Abilene,3.500000
1,Alexandria[m],3.500000
2,Allen,2.500000
3,Amarillo,3.500000
4,Anaheim,3.500000
5,Anchorage[p],3.333333
6,Ann Arbor,3.000000
7,Antioch,3.500000
8,Arlington,3.500000
9,Arvada,3.500000


Merging the citywise_venues_weights to the old wikipedia page table on the basis of City

In [30]:
city_selection=pd.merge(df, citywise_venues_weights_means, on='City')
city_selection=city_selection[['City', 'Population density in Sq-Km2', 'weights']].copy()
city_selection.head()

,City,Population density in Sq-Km2,weights
0,Los Angeles,"3,276/km2",3.5
1,Chicago,"4,600/km2",3.5
2,Houston[3],"1,395/km2",2.5
3,Phoenix,"1,200/km2",3.5
4,Philadelphia[e],"4,511/km2",3.5


Formatting Table

In [31]:
temp_data=city_selection.copy(deep=True)
temp_data['Population density in Sq-Km2']=temp_data['Population density in Sq-Km2'].str.split("/", n=0, expand=True)
temp_data['Population density in Sq-Km2']=temp_data['Population density in Sq-Km2'].str.replace(',','')
temp_data['Population density in Sq-Km2']=temp_data['Population density in Sq-Km2'].astype(float)
city_selection=temp_data.copy(deep=True)
city_selection.head()

,City,Population density in Sq-Km2,weights
0,Los Angeles,3276.0,3.5
1,Chicago,4600.0,3.5
2,Houston[3],1395.0,2.5
3,Phoenix,1200.0,3.5
4,Philadelphia[e],4511.0,3.5


Normalizing the DataFrame

In [32]:
from sklearn import preprocessing
column_names_to_normalize = ['Population density in Sq-Km2', 'weights']
x = city_selection[column_names_to_normalize].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
city_selection[column_names_to_normalize] = pd.DataFrame(x_scaled)
city_selection.head()

,City,Population density in Sq-Km2,weights
0,Los Angeles,0.470174,0.5
1,Chicago,0.664224,0.5
2,Houston[3],0.194489,0.0
3,Phoenix,0.165909,0.5
4,Philadelphia[e],0.651180,0.5


Calculating sum of normalized columns to determine which city has the maximum sum and conclude that only one locality in that city would be best fit

In [33]:
city_selection['sum']=city_selection['Population density in Sq-Km2']+city_selection['weights']
row_num=city_selection['sum'].argmax()
city_name=city_selection['City'].iloc[row_num]
city_name

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  


'Jersey City'

Finding that State in which that city belongs 

In [34]:
row=df.loc[df['City']==city_name].index[0]
state_name=df['State'].iloc[row]
state_name

'New Jersey'

Ramesh wants that the state in which he will open his Sweet shop must have per capita income of minimum of 30000 USD.

In [36]:
income_row=df_state.loc[df_state['State']==state_name].index[0]
per_capita_income=df_state['Per Capita Income'].iloc[income_row]
print("per capita income of {} is ", city_name, per_capita_income)

per capita income of {} is  Jersey City $50,349


As we have concluded that we can choose one location in jersey city, we then check the per capita income of the state. Since it is greater than 30000 USD, it can considered as the best place to open a Indian sweet shop.

Now we will check which location in the jersey city would be best.

In [39]:
lat_newJersey=df['Latitude'].iloc[row]
lan_newJersey=df['Longitude'].iloc[row]
print("Latitude and Longitude are", lat_jersey, lan_jersey)

Latitude and Longitude are 40.7114 -74.0648


Getting all the venues of Jersey city using Foursqaure API within our required radius and LIMIT

In [43]:
def getNearbyVenues1(name, latitudes, longitudes, radius):
    
    LIMIT = 150       
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius, 
            LIMIT)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    print(results)
    venues_list=[]
    venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']
    return(nearby_venues)


new_jersey_venues = getNearbyVenues1(name = 'Jersey City', latitudes = lat_newJersey ,longitudes = lan_newJersey, radius = 2500)
new_jersey_venues.head()

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '566e07e4498e0e75baa42a5e', 'name': 'The Grind Shop', 'location': {'address': '360 Communipaw Ave', 'lat': 40.71167004340248, 'lng': -74.06287219348673, 'labeledLatLngs': [{'label': 'display', 'lat': 40.71167004340248, 'lng': -74.06287219348673}], 'distance': 165, 'postalCode': '07304', 'cc': 'US', 'city': 'Jersey City', 'state': 'NJ', 'country': 'United States', 'formattedAddress': ['360 Communipaw Ave', 'Jersey City, NJ 07304', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-566e07e4498e0e75baa42a5e-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is 

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jersey City,38.3539,-121.9728,The Grind Shop,40.711670,-74.062872,Coffee Shop
1,Jersey City,38.3539,-121.9728,Harry’s Daughter,40.710904,-74.062071,Caribbean Restaurant
2,Jersey City,38.3539,-121.9728,Corgi Spirits at The Jersey City Distillery,40.708304,-74.064803,Distillery
3,Jersey City,38.3539,-121.9728,Hooked JC,40.714709,-74.067009,Fish & Chips Shop
4,Jersey City,38.3539,-121.9728,The Oak On Pine,40.710312,-74.061406,Restaurant


In [44]:
venues_in_newJersey=new_jersey_venues.copy(deep=True)
venues_in_newJersey.shape

(100, 7)

Since we have got 100 categoriesm we are now giving weights for each category for better result

In [45]:
temp = new_jersey_venues.copy(deep = True)
new_weightage_dict= {'Coffee Shop' : 3, 
'Caribbean Restaurant':3,
'Distillery':2,
'Fish & Chips Shop':3,
'Science Museum':3,
'Latin American Restaurant':4,
'Restaurant':5,
'State / Provincial Park':1,
'Diner':1,
'Supermarket':1,
'Bar':1,
'Jazz Club':1,
'Golf Course':3,
'Park':2,
'Cajun / Creole Restaurant':2,
'Bakery':2,
'Go Kart Track':3,
'Taco Place':3,
'Hot Dog Joint':2,
'Food Truck':3,
'Beer Garden':3,
'Boutique':4,
'Café':5,
'Bagel Shop':1,
'Record Shop':1,
'Bakery':1,
'Pizza Place':1,
'Ramen Restaurant':1,
'Wine Bar':3,
'Middle Eastern Restaurant':2,
'French Restaurant':2,
'Theater':2,
'Lounge':3,
'Wine Shop':3,
'Cocktail Bar':2,
'New American Restaurant':3,
'Residential Building (Apartment / Condo)':3,
'Pool':4,
'Burger Joint':5,
'Cheese Shop':1,
'Coffee Shop':1,
'Bagel Shop':1,
'Vietnamese Restaurant':1,
'Portuguese Restaurant':1,
'Ice Cream Shop':3,
'Italian Restaurant':2,
'Gym':2,
'Farmers Market':2,
'Bar':3,
'Pizza Place':3,
'Bakery':2,
'Bookstore':3,
'Bar':3,
'Farmers Market':4,
'Asian Restaurant':5,
'Tea Room':1,
'Donut Shop':1,
'Historic Site':1,
'Gym / Fitness Center':1,
'Café':1,
'Mexican Restaurant':3,
'Plaza':2,
'Gay Bar':2,
'Bar':3,
'College Administrative Building':3,
'Mexican Restaurant':2,
'Bakery':3,
'American Restaurant':3,
'American Restaurant':4,
'American Restaurant':5,
'Café':1,
'New American Restaurant':1,
'Chocolate Shop':1,
'Gym':1,
'Grocery Store':1,
'Middle Eastern Restaurant':3,
'American Restaurant':2,
'Frozen Yogurt Shop':2,
'Japanese Restaurant':2,
'Bar':3,
'Liquor Store':3,
'Ice Cream Shop':2,
'Fish Market':3,
'Indie Movie Theater':3,
'Grocery Store':4,
'Modern European Restaurant':5,
'American Restaurant':1,
'Poke Place':1,
'Ramen Restaurant':1,
'Diner':1,
'Brewery':1,
'Burger Joint':3,
'Burger Joint':2,
'Café':2,
'Fried Chicken Joint':2,
'Beer Garden':3,
'Gym / Fitness Center':3,
'Vietnamese Restaurant':2,
'Italian Restaurant':3,
'Pet Store':3}

Plotting all the venues that we have got from the Foursqaure API

In [46]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [50]:
venues_map=folium.Map(location=[lat_newJersey,lan_newJersey], zoom_start=13)
folium.features.CircleMarker(
    [lat_newJersey, lan_newJersey],
    radius=10,
    popup='Jersey City',
    color='yellow',
    fill=True,
    fil_color='red',
    fill_opacity=0.6
).add_to(venues_map)

Add all the venue of the jersey city to the map as blue circle markers

In [53]:
for lat, lng, label in zip(venues_in_newJersey['Venue Latitude'], venues_in_newJersey['Venue Longitude'], venues_in_newJersey['Venue']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        ).add_to(venues_map)
venues_map

Assigning weights to each category, same as we have given for each city

In [54]:

allVenuesinCity1 = temp['Venue Category']

f_weights1 = []
for i in allVenuesinCity1:
    if i in new_weightage_dict.keys():
        f_weights1.append(new_weightage_dict[i])
    else :
        f_weights1.append(0)
k['weights'] = f_weights1;
k.head()

ValueError: Length of values does not match length of index